<img align="left" src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=130 style="padding: 20px"> 

# Photo-z Compute Scalability Tests
## Optimizing software infrastructure to compute photo-zs in the LSST scale: preparing for LSST DR1.

<br><br>

--- 
Main notebook: [PZ_Compute_Tests.ipynp](./PZ_Compute_Tests.ipynp)

Control spreadsheet: [PZ Compute Runs](https://docs.google.com/spreadsheets/d/1GKlDhLx7oXTjwBXoj8pzfrqnE7X-4nUW2sYDuY-tx94/edit?usp=sharing)

Project members: Julia Gschwend, Heloisa Mengisztki, Cristiano Singulani, Henrique Dante

Last verified run: 27/07/2023

--- 



# Test 6: Test variation with PDF resolution 

Science question: 

_"What is the impact on the pipeline execution speed of changing the output data size? Empirical exploration of  different posterior PDF resolutions."_

Apollo nodes: apl02, apl04, apl06 

Input data: DP0.2 Full (1935 pre-processed parquet files = 278,318,452 objects = 33GB) 

Tested for 4 different options of PDF binning: 31, 101, 201, 301 (default)

In [ ]:
import numpy as np
import pandas as pd
import tables_io
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime 
import time 

%matplotlib inline

In [ ]:
apollo_dict = {'10.148.0.11' : 'apl01', 
                '10.148.0.12' : 'apl02', 
                '10.148.0.13' : 'apl03', 
                '10.148.0.14' : 'apl04', 
                '10.148.0.15' : 'apl05', 
                '10.148.0.16' : 'apl06', 
                '10.148.0.17' : 'apl07', 
                '10.148.0.18' : 'apl08', 
                '10.148.0.19' : 'apl09',                
                '10.148.0.27' : 'apl10', 
                '10.148.0.28' : 'apl11', 
                '10.148.0.29' : 'apl12', 
                '10.148.0.30' : 'apl13', 
                '10.148.0.31' : 'apl14', 
                '10.148.0.32' : 'apl15',
                '10.148.0.26' : 'apl16'} 

Read results collected from htcondor log files and stored in CSV summary files: 

In [ ]:
# FlexZBoost
test_31  = pd.read_csv('results/tests/test_fzboost_31_bins.csv') 
test_101 = pd.read_csv('results/tests/test_fzboost_101_bins.csv')
test_201 = pd.read_csv('results/tests/test_fzboost_201_bins.csv')
test_301 = pd.read_csv('results/tests/test_fzboost_301_bins.csv')

In [ ]:
test_201.host.unique()

In [ ]:
bad_hosts = []
for host, name in apollo_dict.items(): 
    if (name == "apl13") | (name == "apl14") | (name == "apl15"): 
        bad_hosts.append(host) 
        print(host)

Data cleaning: remove results generated by faulty machines (IP hosts above) to minimize bias. 

In [ ]:
query = f'host != "{bad_hosts[0]}" & host != "{bad_hosts[1]}" & host != "{bad_hosts[2]}" '  
test_31.query(query, inplace=True)  
test_101.query(query, inplace=True)  
test_201.query(query, inplace=True)   
test_301.query(query, inplace=True)   

In [ ]:
test_301.host.unique()

Organize dataframes from the tests results used in the analysis: 

In [ ]:
fzboost_runs = {
    'test_31' : test_31,
    'test_101': test_101,
    'test_201': test_201,
    'test_301': test_301
}

In [ ]:
for test, df in fzboost_runs.items():
    print(f'{test} run in {len(df.host.unique())} nodes: ')
    print([apollo_dict[host] for host in df.host.unique()])
    print('---')

Compute speed$^{-1}$ in milliseconds per object and add to each results dataframe: 

In [ ]:
for results_df in fzboost_runs.values():
    results_df['speed'] = (results_df['time_diff']/results_df['chunks'])*1000.

In [ ]:
fzboost_runs['test_31'].head()

Build a dataframe with process summary info:

Fuction to recalculate effective runtime, taking into account only the files processed by the good nodes: 

In [ ]:
def calc_runtime(pz_results_dict, test_name):   
    str_begin = pz_results_dict[test_name]['time_begin'].min()
    str_end = pz_results_dict[test_name]['time_end'].max()
    t_begin = datetime.strptime(str_begin,'%Y-%m-%d %H:%M:%S')
    t_end = datetime.strptime(str_end,'%Y-%m-%d %H:%M:%S')
    dt = (t_end - t_begin)
    runtime = dt.total_seconds()
    return str_begin, str_end, runtime 

example

In [ ]:
test = 'test_31'
begin, end, runtime  = calc_runtime(fzboost_runs, test)
print(f'test {test} starded at {begin}, finished at {end}, and took ~{round(runtime/60.)} minutes')

In [ ]:
fzboost_info = {}
for key in fzboost_runs.keys():
    fzboost_info[key] = {}
for test_name, results_df in fzboost_runs.items():
    hosts = [] 
    for host, name in apollo_dict.items():
        if host in results_df['host'].unique():
            hosts.append(name)
    fzboost_info[test_name]['hosts'] = hosts
    fzboost_info[test_name]['n_cores'] = len(hosts) * 56 
    fzboost_info[test_name]['n_obj'] = np.sum(results_df['chunks'])
    begin, end, runtime  = calc_runtime(fzboost_runs, test_name)
    fzboost_info[test_name]['time_begin'] = begin
    fzboost_info[test_name]['time_end'] = end
    fzboost_info[test_name]['runtime'] = runtime
    fzboost_info[test_name]['n_files'] = len(results_df['host'])
    fzboost_info[test_name]['avg_speed'] = np.average(results_df['speed'])   
    fzboost_info[test_name]['std_speed'] = np.std(results_df['speed'])   
fzboost_info = pd.DataFrame(fzboost_info).T

In [ ]:
fzboost_info.index

In [ ]:
fzboost_info

--- 
## Linear fit

In [ ]:
def calc_runtime_from_summary_table(list_of_string_times):
    times = []
    for t in list_of_string_times:
        delta = datetime.strptime(t, '%H:%M:%S') - datetime(1900, 1, 1, 0, 0, 0)
        times.append(delta.total_seconds())
    return np.array(times)

In [ ]:
len(fzboost_info)

In [ ]:
fzboost_info.runtime

In [ ]:
x = np.array([31, 101, 201, 301]) # PDF binning 
y_estimate = np.array(fzboost_info.runtime) 

In [ ]:
plt.figure(dpi=200)
plt.grid(True)
plt.plot(x, y_estimate/60., 'o-', label="pz estimate")
# plt.xlabel('dataset size (million objects)')
# plt.ylabel('total runtime (min)')
plt.legend(loc = "upper right")
plt.xlabel('number of PDF bins')
plt.ylabel('total runtime (min)')
plt.legend()
plt.tight_layout()
plt.savefig('test_pdf_bins.png')